In [1]:
import pandas as pd
import os
import sys
##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/munch/__init__.py:24: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
SCRIPT_DIR_PATH = os.getcwd()
PARENT_DIR_PATH = os.path.dirname(SCRIPT_DIR_PATH)
TABLEAU_OUTPUT_DIR = os.path.join(PARENT_DIR_PATH, 'Tableau', 'data')
DATA_DIR_PATH = os.path.join(SCRIPT_DIR_PATH, 'data')
SSP_RUN_OUTPUT_DIR = os.path.join(PARENT_DIR_PATH, 'ssp_run_output')
run_id = "2025-08-27T20;12;53.345956"  # Change this to your run ID
RUN_ID_OUTPUT_DIR = os.path.join(SSP_RUN_OUTPUT_DIR, f"sisepuede_summary_results_run_sisepuede_run_{run_id}")

In [3]:
# Save original path
original_sys_path = list(sys.path)

# Add the custom path
custom_path = os.path.abspath("../../data_processing/utils")
sys.path.append(custom_path)

# Import your module
import common_data_needs as cdn

# Revert to original sys.path
sys.path = original_sys_path

/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/sisepuede/utilities/_toolbox.py:533: UserWarning: Path '/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/sisepuede/out/sisepuede_run_2025-08-27T21;32;26.286860' not found. It will not be created.
  warnings.warn(msg)
/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/sisepuede/core/model_attributes.py:6826: UserWarning: 

                        MISSIONSEARCHNOTE: As of 2023-10-06, there is a temporary solution 
                        implemeted in ModelAttributes.get_variable_to_simplex_group_dictionary() 
                        to ensure that transition probability rows are enforced on a simplex.
                        
                        
                        FIX THIS ASAP TO DERIVE PROPERLY.
                        
                        
  warnings.warn(
/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/sisepuede/utilities/_toolbox.py:533: UserWarning: Path '/

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   1081.3 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
/opt/miniconda3/envs/ssp_uganda_env/lib/python3.11/site-packages/sisepuede/utilities/_toolbox.py:2622: UserWarning: Warning passed from optional_log: Successfully initialized JuMP optimizer from solver module HiGHS..
  warnings.warn(f"Warning passed from optional_log: {msg}.")


In [4]:
dict_ssp = cdn._setup_sisepuede_elements()

matt = dict_ssp.get("model_attributes", )
models = dict_ssp.get("models", )
regions = dict_ssp.get("regions", )
time_periods = dict_ssp.get("time_periods", )

In [5]:
ref_year = 2022  # Change this to 2020 or 2022 depending on which year you are processing
df_rescaled = pd.read_csv(os.path.join(TABLEAU_OUTPUT_DIR, 'emissions_uganda_WIDE_INPUTS_OUTPUTS_combined.csv'))
df_map = pd.read_csv(os.path.join(DATA_DIR_PATH, f'emission_targets_uganda_{ref_year}.csv'))
df_original = pd.read_csv(os.path.join(RUN_ID_OUTPUT_DIR, "WIDE_INPUTS_OUTPUTS.csv"))

In [6]:
df_rescaled.head()

,strategy_id,primary_id,Edgar_Class,CSC.Sector,CSC.Subsector,value,Year,Gas,design_id,future_id,strategy,Code,Contry,source,value_original,value_hp
0,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.424422,2022,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.424422,NaN
1,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.426446,2023,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.426446,NaN
2,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.428451,2024,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.428451,NaN
3,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.430436,2025,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.430436,NaN
4,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.432402,2026,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.432402,NaN


In [7]:
df_map.head()

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,ids,UGA
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,1:lvst:ch4,9.103849
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,2:lsmm:ch4,9.103849
2,lsmm,n2o,AG - Livestock:N2O,AG - Livestock,Agriculture,emission_co2e_n2o_lsmm_direct_anaerobic_digest...,3:lsmm:n2o,2.303600
3,agrc,co2,AG - Crops:CO2,AG - Crops,Agriculture,emission_co2e_co2_agrc_biomass_bevs_and_spices...,4:agrc:co2,0.005538
4,agrc,ch4,AG - Crops:CH4,AG - Crops,Agriculture,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,5:agrc:ch4,0.424422


In [8]:
df_original.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,uganda,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,...,2.123306,0.762027,0.982737,0.3674,0.784492,2.755775,76.794300,4.446020,4.002223,92.81
1,0,uganda,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,...,2.171183,0.716171,0.984473,0.3674,0.803826,2.826323,77.459654,4.558346,4.070985,92.81
2,0,uganda,2,750431.308342,2.394373e+06,184455.809571,1.337642e+06,597.343855,607275.938034,3.445560e+06,...,2.136681,0.771066,0.999795,0.3674,0.808390,2.908776,77.263223,5.202683,4.007029,92.81
3,0,uganda,3,754614.187122,2.407719e+06,185483.960027,1.345098e+06,600.673429,610660.873612,3.464766e+06,...,2.134035,0.744831,1.441325,0.3674,0.804774,2.961356,76.784930,5.605283,4.014348,92.81
4,0,uganda,4,759734.779356,2.424057e+06,186742.600192,1.354225e+06,604.749424,614804.640560,3.488277e+06,...,2.172065,0.458120,1.528325,0.3674,0.799814,2.970174,77.726965,4.805888,4.035314,92.81


In [9]:
df_map["Edgar_Subsector"].unique()

array(['AG - Livestock', 'AG - Crops', 'EN - Building',
       'EN - Electricity/Heat', 'EN - Fugitive Emissions',
       'EN - Manufacturing/Construction', 'EN - Transportation',
       'IN - Industrial Processes', 'Waste - Solid Waste',
       'Waste - Wastewater Treatment', 'LULUCF - Forest Land',
       'LULUCF - Deforestation', 'LULUCF - Other Land',
       'LULUCF - Organic Soil', 'CCSQ'], dtype=object)

In [10]:
# data frames to adjust
df_map_entc = df_map[
    df_map["Edgar_Subsector"].isin(["EN - Electricity/Heat"])
]
df_map_fgtv = df_map[
    df_map["Edgar_Subsector"].isin(["EN - Fugitive Emissions"])
]
df_map_waso = df_map[
    df_map["Edgar_Class"].isin(["Waste - Solid Waste:N2O"])
]

In [11]:
_FIELD_EDGAR_CLASS = "Edgar_Class"
_FIELD_PRIMARY = matt.dim_primary_id
_FIELD_STRATEGY = matt.dim_strategy_id
_FIELD_VALUE = "value"
_FIELD_YEAR = "Year"

In [12]:
def agg_from_edgar_subsector(
    df_original: pd.DataFrame,
    df_rescaled: pd.DataFrame,
    df_map: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    
    df_rescaled_out = df_rescaled.copy()

    for i, row in df_map.iterrows():
        
        edgar_class = str(row[_FIELD_EDGAR_CLASS].strip())
        fields = row["Vars"].strip().split(":")
        
        inds_cur = (
            ~df_rescaled_out[_FIELD_STRATEGY].isna()
            & df_rescaled_out[_FIELD_EDGAR_CLASS].isin([edgar_class])
        )
        
        inds_other = ~inds_cur

        # get data frame component to adjust and original value
        df_adjust = df_rescaled_out[inds_cur]
        #df_adjust[_FIELD_STRATEGY] = df_adjust[_FIELD_STRATEGY].astype(int)
        df_adjust[_FIELD_PRIMARY] = df_adjust[_FIELD_PRIMARY].astype(int)
        
        # extract totals 
        df_total = (
            time_periods
            .tps_to_years(
                df_original[[_FIELD_PRIMARY, time_periods.field_time_period]].copy()
            )
            .drop(columns = [time_periods.field_time_period])
        )
        
        df_total[_FIELD_VALUE] = df_original[fields].sum(axis = 1, )
        df_total = (
            df_total
            .rename(
                columns = {
                    time_periods.field_year: _FIELD_YEAR,
                }
            )
        )
        
        # overwrite with totals
        df_adjust_out = sf.match_df_to_target_df(
            df_adjust,
            df_total,
            [_FIELD_PRIMARY, _FIELD_YEAR],
            
        )

        df_rescaled_out = sf._concat_df(
            [
                df_rescaled_out[inds_other],
                df_adjust_out,
            ]
        )
        
        

    return df_rescaled_out
# 

In [13]:
field = "Edgar_Class"

In [14]:
dfs_iter = [
    df_map_entc,
    df_map_fgtv,
    df_map_waso
]

for df in dfs_iter:
    # 
    df_rescaled_agg = agg_from_edgar_subsector(
        df_original.copy(),
        df_rescaled.copy(),
        df,
    )

    df_rescaled_out = df_rescaled_agg.dropna(subset=[_FIELD_PRIMARY])
    df_rescaled_out[[_FIELD_PRIMARY, _FIELD_STRATEGY]] = df_rescaled_out[[_FIELD_PRIMARY, _FIELD_STRATEGY]].astype(int)

    df_rescaled_out_sorted = (
        df_rescaled_out
        .dropna(subset=[_FIELD_PRIMARY])
        .sort_values(
            by=[
                _FIELD_PRIMARY,
                field,
                _FIELD_YEAR,
            ]
        )
    )

/var/folders/tc/7lgly1g56n1cclb6ls7rcwzr0000gn/T/ipykernel_3455/3014047980.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust[_FIELD_PRIMARY] = df_adjust[_FIELD_PRIMARY].astype(int)
/var/folders/tc/7lgly1g56n1cclb6ls7rcwzr0000gn/T/ipykernel_3455/3014047980.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust[_FIELD_PRIMARY] = df_adjust[_FIELD_PRIMARY].astype(int)
/var/folders/tc/7lgly1g56n1cclb6ls7rcwzr0000gn/T/ipykernel_3455/3014047980.py:26: SettingWithCopyWarning: 
A value is trying

In [15]:
df_rescaled_out_sorted

,strategy_id,primary_id,Edgar_Class,CSC.Sector,CSC.Subsector,value,Year,Gas,design_id,future_id,strategy,Code,Contry,source,value_original,value_hp
0,0,0,AG - Crops:CH4,Agriculture,AG - Crops,0.424422,2022,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.424422,NaN
1,0,0,AG - Crops:CH4,Agriculture,AG - Crops,0.426446,2023,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.426446,NaN
2,0,0,AG - Crops:CH4,Agriculture,AG - Crops,0.428451,2024,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.428451,NaN
3,0,0,AG - Crops:CH4,Agriculture,AG - Crops,0.430436,2025,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.430436,NaN
4,0,0,AG - Crops:CH4,Agriculture,AG - Crops,0.432402,2026,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.432402,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6739,6006,73073,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,1.123727,2066,N2O,0.0,0.0,NDC_2,UGA,uganda,SISEPUEDE,1.123727,NaN
6740,6006,73073,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,1.126803,2067,N2O,0.0,0.0,NDC_2,UGA,uganda,SISEPUEDE,1.126803,NaN
6741,6006,73073,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,1.129626,2068,N2O,0.0,0.0,NDC_2,UGA,uganda,SISEPUEDE,1.129626,NaN
6742,6006,73073,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,1.132135,2069,N2O,0.0,0.0,NDC_2,UGA,uganda,SISEPUEDE,1.132135,NaN


In [16]:
df_rescaled.strategy.unique()

array(['Strategy TX:BASE', 'NZ', 'Historical', 'NDC_2'], dtype=object)

In [17]:
# Filter df_rescaled to only include rows where strategy is "Historical"
df_rescaled_historical = df_rescaled[df_rescaled["strategy"] == "Historical"]
df_rescaled_historical.head()

,strategy_id,primary_id,Edgar_Class,CSC.Sector,CSC.Subsector,value,Year,Gas,design_id,future_id,strategy,Code,Contry,source,value_original,value_hp
4018,NaN,NaN,AG - Crops:CH4,Agriculture,AG - Crops,0.8587,1995,CH4,NaN,NaN,Historical,UGA,uganda,EDGAR,0.8587,NaN
4019,NaN,NaN,AG - Crops:CH4,Agriculture,AG - Crops,0.8587,1996,CH4,NaN,NaN,Historical,UGA,uganda,EDGAR,0.8587,NaN
4020,NaN,NaN,AG - Crops:CH4,Agriculture,AG - Crops,0.8587,1997,CH4,NaN,NaN,Historical,UGA,uganda,EDGAR,0.8587,NaN
4021,NaN,NaN,AG - Crops:CH4,Agriculture,AG - Crops,0.8587,1998,CH4,NaN,NaN,Historical,UGA,uganda,EDGAR,0.8587,NaN
4022,NaN,NaN,AG - Crops:CH4,Agriculture,AG - Crops,0.8587,1999,CH4,NaN,NaN,Historical,UGA,uganda,EDGAR,0.8587,NaN


In [18]:
# Concat df_rescaled_historical with df_rescaled_out_sorted
df_rescaled_out_sorted = pd.concat([df_rescaled_out_sorted, df_rescaled_historical], ignore_index=True)
df_rescaled_out_sorted

,strategy_id,primary_id,Edgar_Class,CSC.Sector,CSC.Subsector,value,Year,Gas,design_id,future_id,strategy,Code,Contry,source,value_original,value_hp
0,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.424422,2022,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.424422,NaN
1,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.426446,2023,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.426446,NaN
2,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.428451,2024,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.428451,NaN
3,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.430436,2025,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.430436,NaN
4,0.0,0.0,AG - Crops:CH4,Agriculture,AG - Crops,0.432402,2026,CH4,0.0,0.0,Strategy TX:BASE,UGA,uganda,SISEPUEDE,0.432402,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,NaN,NaN,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,0.485827,2018,N2O,NaN,NaN,Historical,UGA,uganda,EDGAR,0.485827,NaN
6889,NaN,NaN,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,0.498912,2019,N2O,NaN,NaN,Historical,UGA,uganda,EDGAR,0.498912,NaN
6890,NaN,NaN,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,0.505633,2020,N2O,NaN,NaN,Historical,UGA,uganda,EDGAR,0.505633,NaN
6891,NaN,NaN,Waste - Wastewater Treatment:N2O,Waste,Waste - Wastewater Treatment,0.513311,2021,N2O,NaN,NaN,Historical,UGA,uganda,EDGAR,0.513311,NaN


In [19]:
df_rescaled_out_sorted["CSC.Subsector"].unique()

array(['AG - Crops', 'AG - Livestock', 'CCSQ', 'EN - Building',
       'EN - Electricity/Heat', 'EN - Fugitive Emissions',
       'EN - Manufacturing/Construction', 'EN - Transportation',
       'IN - Industrial Processes', 'LULUCF - Deforestation',
       'LULUCF - Forest Land', 'LULUCF - Organic Soil',
       'LULUCF - Other Land', 'Waste - Solid Waste',
       'Waste - Wastewater Treatment'], dtype=object)

In [20]:
# Make sure fgtv is all zero
df_rescaled_out_sorted.loc[
    df_rescaled_out_sorted["CSC.Subsector"] == 'EN - Fugitive Emissions',
    ["value", "value_original"]
] = 0

In [21]:
print(df_rescaled_out_sorted.shape)
print(df_rescaled.shape)

(6893, 16)
(6893, 16)


In [22]:

df_rescaled_out_sorted.to_csv(
        os.path.join(TABLEAU_OUTPUT_DIR, f"uganda_rescaled_{run_id}_overwritten_{ref_year}.csv"),
        index=None,
        encoding="UTF-8",
    )